La API de MercadoLibre hace uso de algo que se llama Resource o Recurso, que viene del protocolo HTTP. Mediante un string, podemos identificar el o los recursos que queremos consultarle a MercadoLibre.

Como nosotros queremos consultar Casas o Deparamentos que se encuentren en Venta, bastaría con hacer una peticion a la API con el siguente Recurso: *sites/MLA/search?category=MLA1459&PROPERTY_TYPE=242060,242062&OPERATION=242075*

El inconveniente es que MercadoLibre, nos permite obtener 10.000 registros, dado un recurso determinado. Es decir, que haciendo esta query solo podríamos armar un dataset de 10.000 registros.

Como podemos solucionarlo? Haciendo mas especifico el recurso, es decir, pidiendo casas o departamentos, que esten en venta, en la ciudad de buenos aires, en el barrio villa crespo.
Y para ese recurso especifico, vamos a poder obtener 10.000 registros.

Y eso es lo que hace esta notebook. Mediante querys a MercadoLibre, obeniendo los filtros que tenemos disponibles para hacer cada vez mas especifico el recurso, arma la lista de resources, para que luego otra notebook se encargue de realizar las peticiones correspondientes.

Puntualmente, vamos a especificar los resurces con una Ciudad, y un Barrio. 
Buenos Aires, Recoleta
Buenos Aires, Almagro
Pilar, ..
Pilar, ..

Instalamos una librería para facilitar el uso de la API de MercadoLibre

In [1]:
pip install git+https://github.com/mercadolibre/python-sdk.git

  Cloning https://github.com/mercadolibre/python-sdk.git to /tmp/pip-req-build-fm84s95m
  Running command git clone -q https://github.com/mercadolibre/python-sdk.git /tmp/pip-req-build-fm84s95m
     |████████████████████████████████| 138 kB 5.0 MB/s 
  Created wheel for meli: filename=meli-3.0.0-py3-none-any.whl size=39713 sha256=8df79f3b127f3320049d9d72184b100565018ee322562c98e89cd9f066001874
  Stored in directory: /tmp/pip-ephem-wheel-cache-36f8phe1/wheels/60/64/2d/4c5c8a03c8a655c3d499ecd487a15dc2e740aed5d6574b6305
Successfully built meli
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
da

Encapsulamos la funcion get de MercadoLibre, con algunos parametros de utilidad.

Offset es el indice del primer registro que queremos consultar. Ej: Si es 0, nos traerá el primer registro que la API tiene disponible para el Resource que estamos consultando. 

Limit es la cantidad de registros que nos traerá la API. El maximo es 50.

In [2]:
def get_request(resource, offset=0, limit=50):
    api_instance = meli.RestClientApi(api_client)
    try:
        return api_instance.resource_get(resource+"&offset="+str(offset)+"&limit="+str(limit), access_token)
    except ApiException as e:
      print("Exception when calling RestClientApi->resource_get: %s\n" % e)

MercadoLibre, en todas las respuestas que hace la API, contesta además los filtros disponibles que se pueden agregar para seguir haciendo más especifica la query. Esta funcion, dado una key de un filtro que deseamos, lo obtiene de la lista de filtros desponibles, de una respuesta determinada. En la siguiente función, veremos un ejemplo de esto.

In [3]:
def get_values(response, key):
  for filter in response["available_filters"]:
    if filter["id"]==key:
      return filter["values"]
  return None

Si ya estamos filtrando por una ciudad determinada, el filtro de "Ciudad" no va a aparecer en los filtros disponibles.
Esta funcion verifica si un filtro (Ciudad o Barrio) se encuentra o no en los filtros disponibles.

In [4]:
def contains(response, key):
  if get_values(response, key) != None:
    return True
  return False

Como para especificar en el recurso, necesitamos los ID de cada ciudad, no podemos indicar City=BuenosAires, esta funcion obtiene, de la lista de filtros disponibles, el ID de todas las ciudades que se encuentren disponibles en la respuesta.

In [5]:
def get_city_ids(api_response):
  city_ids = []
  for city in get_values(api_response, "city"):
    city_ids.append(city["id"])

  return city_ids

Lo mismo que la funcion anterior, pero para los barrios.

In [6]:
def get_neighborhood_ids(api_response):
  neighborhood_ids = []
  for neighborhood in get_values(api_response, "neighborhood"):
    neighborhood_ids.append(neighborhood["id"])

  return neighborhood_ids

Para autenticarnos con la API, se debe acceder al siguiente [link](https://auth.mercadolibre.com.ar/authorization?response_type=code&client_id=96683996985285) , copiar el codigo que aparece en la URL, pegarlo en la celda de abajo, donde dice "code", y ejecutar la celda, para obtener el token, que será el que utilizaremos en cada peticion a la API.


In [7]:
from __future__ import print_function
import time
import meli
from meli.rest import ApiException
from pprint import pprint

with meli.ApiClient() as api_client:
    api_instance = meli.OAuth20Api(api_client)
    grant_type = 'authorization_code'
    client_id = '96683996985285'
    client_secret = 'nMeP0YOMz9ZW0ujUdp9MEdV1Spr23vWR'
    redirect_uri = 'https://www.google.com'
    code = 'TG-61ca712414c25c001a222278-204954233'
    refresh_token = 'refresh_token_example'
try:
    api_response = api_instance.get_token(grant_type=grant_type, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, code=code, refresh_token=refresh_token)
    access_token = api_response["access_token"]
except ApiException as e:
    print("Exception when calling OAuth20Api->get_token: %s\n" % e)

Realizamos una primer peticion a la API, filtrando por Casas o Departamentos, en Venta.

In [8]:
api_response = get_request('sites/MLA/search?category=MLA1459&offset=3&PROPERTY_TYPE=242060,242062&OPERATION=242075', 0, 0)
api_response

{'available_filters': [{'id': 'official_store',
   'name': 'Tiendas oficiales',
   'type': 'text',
   'values': [{'id': 'all',
     'name': 'Todas las tiendas oficiales',
     'results': 24009},
    {'id': '2743', 'name': 'Sistema Coldwell Banker', 'results': 1556},
    {'id': '2747', 'name': 'Veronica Espinosa Propiedades', 'results': 1121},
    {'id': '2536', 'name': 'Century 21', 'results': 1807},
    {'id': '2636', 'name': 'Toribio Achaval', 'results': 1635},
    {'id': '2707', 'name': 'Tizado', 'results': 689},
    {'id': '2761', 'name': 'Prisma Propiedades', 'results': 1}]},
  {'id': 'state',
   'name': 'Ubicación',
   'type': 'text',
   'values': [{'id': 'TUxBUENBUGw3M2E1',
     'name': 'Capital Federal',
     'results': 94798},
    {'id': 'TUxBUEdSQWU4ZDkz',
     'name': 'Bs.As. G.B.A. Norte',
     'results': 66228},
    {'id': 'TUxBUEdSQXJlMDNm', 'name': 'Bs.As. G.B.A. Sur', 'results': 54015},
    {'id': 'TUxBUEdSQWVmNTVm',
     'name': 'Bs.As. G.B.A. Oeste',
     'results': 5

Como estamos solicitando un Offset de 0 y un Limit de 0, no nos trae ningún registro, sino unicamente filtros disponibles, ordenamientos disponibles, los filtros que estamos aplicando actualmente, paginación, y resultados, que está vacio, como dijimos.

Si vemos el available filter con id "State", observamos que tiene valores tales como, Buenos Aires sur, Catamarca, Brasil, etc. Y cada uno de estos nombres, tiene asociado un ID, similar a TUxBUE5FVW4xMzMzNQ. Y luego, la cantidad de registros que contiene ese filtro.

Si nos centramos en este: {'id': 'TUxBUENPUmFkZGIw', 'name': 'Córdoba', 'results': 28815}
Vemos que es Córdoba, con el id TUxBUENPUmFkZGIw, y tiene 28815 registros. Pero nosotros solo podemos consultar 10000 de estos. Por eso es necesario, que no solo apliquemos este filtro, sino otros, mas especificos todavía, como por ejemplo, filtrando por Barrio, para poder lograr consultar la mayor cantidad de registros que podamos.

Es decir, lo que necesitamos es obtener todos esos ID, de los State que nos interesan, para luego en el resource, poder especificar "state=ID_OBTENIDO"

In [9]:
resources_with_state = []
for state in get_values(api_response, "state"):
  if (state["name"] not in("USA","Brasil","Uruguay") ) :
    resources_with_state.append('sites/MLA/search?category=MLA1459&PROPERTY_TYPE=242060,242062&OPERATION=242075&state='+state["id"])

Lo comentado anteriormente, es lo que intenta resolver esta función. Obtiene de la respuesta de la api, los valores que corresponden a la clave "state". Y si no es USA, Brasil o Uruguay, arma el resource más especifico que tenemos hasta el momento, que son Inmuebles, que sean Casas o Departamentos, en Venta, en cada uno de los State que conseguimos, y los guarda en una lista.

Ahora bien, vimos que debemos hacer más especificas las peticiones, porque no llegabamos a consultar todos los registros, como el ejemplo de Cordoba.

Pero acá nos encontramos con un inconveniente a resolver.
Hay Estados que tienen directamente Barrios (Estado > Barrio)
Y hay  Estados, que tienen Ciudades, que tienen Barrios. (Estate > Ciudad > Barrio)

Por eso, no podemos tratar a todas las respuestas de la api por igual. Porque si se trata de una respuesta, que corresponde al caso 2, y nosotros le pedimos de la lista de filtros disponibles, los barrios, va a tirar error, porque no está disponible ese filtro, sino que primero tenemos que filtar por ciudad.

Y eso es lo que hace esta funcion. Si la respuesta contiene "neighborhood" en los filtros disponibles, se trata del caso 1, (se tratará de Buenos Aires, que ya es una ciudad)

Si la respuesta contiene "city" se trata del caso 2, como por ejemplo, Mendoza, que no tiene barrios en si misma, sino que tiene ciudades, que tienen barrios.

In [10]:
resources_with_city = []
resources_with_neighborhood = []

for resource_with_state in resources_with_state:
  api_response = get_request(resource_with_state, 0, 0)
  if contains(api_response, "neighborhood"):
    neighborhood_ids = get_neighborhood_ids(api_response)
    for neighborhood_id in neighborhood_ids:
      resources_with_neighborhood.append(resource_with_state+"&neighborhood="+neighborhood_id)
  else:
    if contains(api_response, "city"):
      city_ids = get_city_ids(api_response)
      for city_id in city_ids:
        resources_with_city.append(resource_with_state+"&city="+city_id)
    else:
      print("No tenia ni neighborhood ni city")


En este punto, hay resources que ya logramos constriurlos lo más especificos que podíamos, los que ya les agregamos el filtro de "neighborhood", que la cantidad la vemos en la siguiente celda.

Pero todavía tenemos que seguir trabajando con los que solo pudimos agregarles el filtro de "city", que la cantidad la vemos mas adelante.

In [11]:
len(resources_with_neighborhood)

62

In [12]:
len(resources_with_city)

392

In [13]:
for resource_with_city in resources_with_city:
  api_response = get_request(resource_with_city, 0, 0)
  if contains(api_response, "neighborhood"):
    neighborhood_ids = get_neighborhood_ids(api_response)
    for neighborhood_id in neighborhood_ids:
      resources_with_neighborhood.append(resource_with_city+"&neighborhood="+neighborhood_id)
  else:
    resources_with_neighborhood.append(resource_with_city)

Si el ya pudimos filtrar por ciudad, intentamos filtar por barrio. En caso de que la respuesta de la API, contenga ese filtro disponible, lo incluimos en la lista de "resources_with_neighborhood" de lo contrario, lo agregamos unicamente filtrando por ciudad.

Si vemos la cantidad de resources que conseguimos, son:

In [14]:
len(resources_with_neighborhood)

2577

Utilizamos pickle, para escribir la lista de resources, en un archivo, en formato binario.

In [15]:
import pickle

file_name = "resources.pkl"

open_file = open(file_name, "wb")

pickle.dump(resources_with_neighborhood, open_file)

open_file.close()

Por ultimo, vamos a subir este archivo a un bucket, de Google Cloud Storage, para eso declaramos dos variables, una con el nombre del proyecto y la otra con el nombre del bucket, y nos autenticamos.

In [16]:
project_id = 'cryptic-opus-335323'
bucket_name = 'bdm-unlu'
from google.colab import auth
auth.authenticate_user()

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Seteamos el proyecto actual de gcloud, utilizando la variable declarada anteriormente.

In [17]:
!gcloud config set project {project_id}

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



Y finalmente subimos el archivo al path bdm-unlu/resources/resources.pkl, para poder consumirlo luego desde otro colab, especificamente, el colab de [Querys](https://colab.research.google.com/drive/157aPMMoghsRTnaZY3f-BqwLCb5zwjs-N#scrollTo=H6rugUtu-uXe).

In [18]:
!gsutil cp resources.pkl gs://{bucket_name}/resources/

Copying file://resources.pkl [Content-Type=application/octet-stream]...
/ [1 files][427.8 KiB/427.8 KiB]                                                
Operation completed over 1 objects/427.8 KiB.                                    
